<a href="https://colab.research.google.com/github/veydantkatyal/llm-crypto-agent/blob/main/crypto_data_analyst_llm_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INSTALL DEPENDENCIES**

In [1]:
!pip -qqq install pip --progress-bar off
!pip -qqq install 'crewai[tools]'==0.28.8 --progress-bar off
!pip -qqq install langchain-groq==0.1.3 --progress-bar off
!pip install exa-py

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.6.1 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.5.1+cu124 requires nvidia-

# **Import Libraries and Set Up API Keys**

In [2]:
from exa_py import Exa
import requests
import pandas as pd
import os

os.environ["GROQ_API_KEY"] = input("Enter your GROQ API Key: ")

Enter your GROQ API Key: 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


# **FETCH CRYPTO NEWS**

In [6]:
os.environ["EXA_API_KEY"] = input("Enter your EXA API Key: ")
exa = Exa(api_key=os.environ["EXA_API_KEY"])
def get_cryptocurrency_news(query):
    result = exa.search_and_contents(query, summary=True)
    # Ensure results exist before iterating
    if result.results:
        news_list = []
        for item in result.results:
            # Extracting attributes directly from the Result object
            news_item = {
                "title": item.title if hasattr(item, "title") else "No Title",
                "url": item.url if hasattr(item, "url") else "#",  # URL and ID are the same
                "id": item.id if hasattr(item, "id") else "#",  # ID is same as URL
                "score": item.score if hasattr(item, "score") else "No Score",
                "published_date": item.published_date if hasattr(item, "published_date") else "Unknown Date",
                "author": item.author if hasattr(item, "author") else "Unknown Author",
                "image": item.image if hasattr(item, "image") else "No Image",
                "favicon": item.favicon if hasattr(item, "favicon") else "No Favicon",
                "summary": item.summary if hasattr(item, "summary") else "No Summary",
                "highlights": item.highlights if hasattr(item, "highlights") else "No Highlights",
                "highlight_scores": item.highlight_scores if hasattr(item, "highlight_scores") else "No Highlight Scores",
            }
            news_list.append(news_item)
        return news_list
    else:
        return []  # Return empty list if no results found

Enter your EXA API Key: 59ad6c19-e945-44e6-9d27-5f2f7893fd3c


 # **Display Top Cryptocurrency News Articles**

In [8]:
# Fetch and display news
news = get_cryptocurrency_news("Bitcoin cryptocurrency")
for article in news[:15]:  # Display top 15 article
    print(f"Title: {article['title']}\nURL: {article['url']}\nID: {article['id']}\nPublished Date: {article['published_date']}\nAuthor: {article['author']}\nImage: {article['image']}\nFavicon: {article['favicon']}\nSummary: {article['summary']}\n")

Title: How does Bitcoin work? - Bitcoin
URL: https://bitcoin.org/en/how-it-works
ID: https://bitcoin.org/en/how-it-works
Published Date: 2024-01-01T00:00:00.000Z
Author: 
Image: https://bitcoin.org/img/icons/opengraph.png?1716491272
Favicon: https://bitcoin.org/favicon.png?1716491272
Summary: Bitcoin is a cryptocurrency that uses a shared public ledger called the blockchain to record all transactions.  Users interact with Bitcoin using wallets which generate addresses for receiving and sending Bitcoin.  Transactions are secured using private keys and signatures, preventing alteration.  The process of confirming transactions and adding them to the blockchain is called mining, a distributed consensus system that ensures the integrity and chronological order of the blockchain.  New users can start using Bitcoin without deep technical knowledge by simply installing a wallet.


Title: Bitcoin - Open source P2P money
URL: https://bitcoin.org/en/
ID: https://bitcoin.org/en/
Published Date: 20

# **Define a Function to Get Daily Closing Prices for a Cryptocurrency**

In [9]:
import os
import requests
import pandas as pd

def get_daily_closing_prices(ticker) -> pd.DataFrame:
    api_key = os.environ.get("ALPHAVANTAGE_API_KEY")
    url = f"https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol={ticker}&market=USD&apikey={api_key}"
    response = requests.get(url)
    data = response.json()

    price_data = data["Time Series (Digital Currency Daily)"]
    daily_close_prices = {
        date: prices["4. close"] for (date, prices) in price_data.items()
    }
    df = pd.DataFrame.from_dict(daily_close_prices, orient="index", columns=["price"])
    df.index = pd.to_datetime(df.index)
    df["price"] = pd.to_numeric(df["price"])
    return df

# **Format the Daily Closing Prices for Display**

In [10]:
def format_prices(price_df: pd.DataFrame) -> str:
    text_output = [
        f"{date.strftime('%Y-%m-%d')} - {row['price']:.2f}"
        for date, row in price_df.head(10).iterrows()
    ]
    return "\n".join(text_output)

# **Display Formatted Bitcoin Price Data**

In [12]:
price_df = get_daily_closing_prices("BTC")
formatted_prices = format_prices(price_df)
print("\nBitcoin Daily Closing Prices:\n", formatted_prices)


Bitcoin Daily Closing Prices:
 2025-03-13 - 83603.79
2025-03-12 - 83659.43
2025-03-11 - 82914.51
2025-03-10 - 78544.71
2025-03-09 - 80699.17
2025-03-08 - 86206.69
2025-03-07 - 86756.98
2025-03-06 - 89921.85
2025-03-05 - 90603.73
2025-03-04 - 87249.96


# **Define Custom Tools for LangChain Agents**

In [13]:
from crewai_tools import tool
from langchain.agents import load_tools
@tool("search_tool")
def search_tool(query: str) -> str:
    """Search for the latest news and provide a summary about a given query using Exa."""
    result = exa.search_and_contents(query, summary=True)
    # Ensure results exist before processing
    if result.results:
        news_list = []
        for item in result.results:
            # Extracting attributes directly from the Result object
            news_item = {
                "title": item.title if hasattr(item, "title") else "No Title",
                "url": item.url if hasattr(item, "url") else "#",  # URL and ID are the same
                "id": item.id if hasattr(item, "id") else "#",  # ID is same as URL
                "score": item.score if hasattr(item, "score") else "No Score",
                "published_date": item.published_date if hasattr(item, "published_date") else "Unknown Date",
                "author": item.author if hasattr(item, "author") else "Unknown Author",
                "image": item.image if hasattr(item, "image") else "No Image",
                "favicon": item.favicon if hasattr(item, "favicon") else "No Favicon",
                "summary": item.summary if hasattr(item, "summary") else "No Summary",
                "highlights": item.highlights if hasattr(item, "highlights") else "No Highlights",
                "highlight_scores": item.highlight_scores if hasattr(item, "highlight_scores") else "No Highlight Scores",
            }
            news_list.append(news_item)
        output = []
        for news_item in news_list:
            output.append(f"Title: {news_item['title']}\nURL: {news_item['url']}\nSummary: {news_item['summary']}\n")
        return "\n".join(output)
    else:
        return "No results found."

# **Define Other LangChain Tools**

In [15]:
@tool("price tool")
def cryptocurrency_price_tool(ticker_symbol: str) -> str:
    """Get daily closing price for a given cryptocurrency ticker symbol for the previous 60 days"""
    price_df = get_daily_closing_prices(ticker_symbol)
    text_output = []

    for date, row in price_df.head(60).iterrows():
        text_output.append(f"{date.strftime('%Y-%m-%d')} - {row['price']:.2f}")
    return "\n".join(text_output)

@tool("search tool")
def cryptocurrency_news_tool(ticker_symbol: str) -> str:
    """Searches for cryptocurrency news using the search_tool.""" # Added docstring
    return search_tool.run(ticker_symbol + " cryptocurrency")

# **Llama3 with Groq for Inference**

In [16]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
# Llama 3 with Groq for inference
llm = ChatGroq(temperature=0, model_name="llama3-70b-8192")
system_message = "You are an experienced Machine Learning & AI Engineer."
human_message = "How to increase inference speed for a 7B LLM?"
prompt = ChatPromptTemplate.from_messages([("system", system_message), ("human", human_message)])
chain = prompt | llm
response = chain.invoke({"text": human_message})
print("\nLLM Response:\n", response.content)


LLM Response:
 The quest for speed!

Increasing inference speed for a 7B Large Language Model (LLM) is crucial for many applications, such as real-time language translation, chatbots, and text summarization. Here are some strategies to help you achieve faster inference speeds:

1. **Model Pruning**: Remove redundant or unnecessary weights and connections in the model to reduce its computational requirements. This can lead to a significant reduction in inference time without sacrificing accuracy. You can use techniques like magnitude-based pruning, L1 regularization, or iterative pruning.
2. **Knowledge Distillation**: Train a smaller, simpler model (the student) to mimic the behavior of the large, pre-trained model (the teacher). The student model can be designed to be more efficient while maintaining similar accuracy. This technique can reduce the model size and inference time.
3. **Quantization**: Represent the model's weights and activations using fewer bits (e.g., int8 instead of 

# **Define LangChain Agents and Tasks**

In [21]:
from crewai import Agent, Crew, Process, Task
from crewai_tools import Tool
human_tools = [
    Tool(
        name="Search",
        func=search_tool.run,
        description="useful for when you need to answer questions about current events or the current state of the world",
    )
]
customer_communicator = Agent(
    role="Senior cryptocurrency customer communicator",
    goal="Find which cryptocurrency the customer is interested in",
    backstory="""You're highly experienced in communicating about cryptocurrencies
    and blockchain technology with customers and their research needs""",
    verbose=True,
    allow_delegation=False,
    llm=llm,
    max_iter=5,
    memory=True,
    tools=human_tools,
)
news_analyst = Agent(
    role="Cryptocurrency News Analyst",
    goal="""Get news for a given cryptocurrency. Write 1 paragraph analysis of
    the market and make prediction - up, down or neutral.""",
    backstory="""You're an expert analyst of trends based on cryptocurrency news.
    You have a complete understanding of macroeconomic factors, but you specialize
    into analyzing news.
    """,
    verbose=True,
    allow_delegation=False,
    llm=llm,
    max_iter=5,
    memory=True,
    tools=[cryptocurrency_news_tool],
)

price_analyst = Agent(
    role="Cryptocurrency Price Analyst",
    goal="""Get historical prices for a given cryptocurrency. Write 1 paragraph analysis of
    the market and make prediction - up, down or neutral.""",
    backstory="""You're an expert analyst of trends based on cryptocurrency
    historical prices. You have a complete understanding of macroeconomic factors,
    but you specialize into technical analysis based on historical prices.

    """,
    verbose=True,
    allow_delegation=False,
    llm=llm,
    max_iter=5,
    memory=True,
    tools=[cryptocurrency_price_tool],
)

writer = Agent(
    role="Cryptocurrency Report Writer",
    goal="""Write 1 paragraph report of the cryptocurrency market.""",
    backstory="""
    You're widely accepted as the best cryptocurrency analyst that

    understands the market and have tracked every asset for more than 10 years. Your trends
    analysis are always extremely accurate.

    You're also master level analyst in the traditional markets and have deep understanding

    of human psychology. You understand macro factors and combine those multiple

    theories - e.g. cycle theory. You're able to hold multiple opinions when analysing anything.

    You understand news and historical prices, but you look at those with a

    healthy dose of skepticism. You also consider the source of news articles.

    Your most well developed talent is providing clear and concise summarization

    that explains very complex market topics in simple to understand terms.

    Some of your writing techniques include:

    - Creating a bullet list (executive summary) of the most important points

    - Distill complex analyses to their most important parts

    You writing transforms even dry and most technical texts into

    a pleasant and interesting read.""",

    llm=llm,
    verbose=True,
    max_iter=5,
    memory=True,
    allow_delegation=False,
)

# **Tasks**

# **Define a Crew to Manage the Agents and Tasks**

In [22]:
get_cryptocurrency = Task(

    description=f"Ask which cryptocurrency the customer is interested in.",

    expected_output="""Cryptocurrency symbol that the human wants you to research e.g. BTC.""",

    agent=customer_communicator,

)

from datetime import datetime

get_news_analysis = Task(

    description=f"""

    Use the search tool to get news for the cryptocurrency

    The current date is {datetime.now()}.

    Compose the results into a helpful report""",

    expected_output="""Create 1 paragraph report for the cryptocurrency,

    along with a prediction for the future trend

    """,

    agent=news_analyst,

    context=[get_cryptocurrency],

)

get_price_analysis = Task(

    description=f"""

    Use the price tool to get historical prices

    The current date is {datetime.now()}.

    Compose the results into a helpful report""",

    expected_output="""Create 1 paragraph summary for the cryptocurrency,

    along with a prediction for the future trend

    """,

    agent=price_analyst,

    context=[get_cryptocurrency],

)

write_report = Task(

    description=f"""Use the reports from the news analyst and the price analyst to

    create a report that summarizes the cryptocurrency""",

    expected_output="""1 paragraph report that summarizes the market and

    predicts the future prices (trend) for the cryptocurrency""",

    agent=writer,

    context=[get_news_analysis, get_price_analysis],

)

In [23]:
crew = Crew(
    agents=[customer_communicator, price_analyst, news_analyst, writer],
    tasks=[get_cryptocurrency, get_news_analysis, get_price_analysis, write_report],
    verbose=2,
    process=Process.sequential,
    full_output=True,
    share_crew=False,
    manager_llm=llm,
    max_iter=15,
)

results = crew.kickoff()

 [DEBUG]: == Working Agent: Senior cryptocurrency customer communicator
 [INFO]: == Starting Task: Ask which cryptocurrency the customer is interested in.


> Entering new CrewAgentExecutor chain...
Thought: I need to ask the customer which cryptocurrency they are interested in to provide the best possible assistance.

Action: None (no action needed, just ask the question)

Action Input: None
 

Action 'None (no action needed, just ask the question)' don't exist, these are the only available Actions: Search: Search() - useful for when you need to answer questions about current events or the current state of the world

Thought: I need to ask the customer which cryptocurrency they are interested in to provide the best possible assistance.

Action: Search

Action Input: {"query": "Which cryptocurrency would you like to know more about?"}
Using Tool: search_tool
Using Tool: search_tool
 

Title: Chia Network - XCH - Build on A Better Blockchain
URL: https://www.chia.net
Summary: The webpag